In [3]:
FWD = 1
BWD = 0
PAR_OPEN = "["
PAR_CLOSE = "]"
class DAY_18:
    def __init__(self, string, verbose=True):
        self.__string = string
        self.__verbose = verbose
        if self.__verbose: print(self.__string)
        
    def convert_snailfish_to_array(self):
        '''
        Wandle den String in ein Array um. Dabei werden die Klammern bewahrt, gleichzeitig Zahlen größer als 9 als einzelne Zahl abgespeichert
        '''
        integers = [str(i) for i in range(10)]
        self.__snailfish_array = []
        i = 0
        while i < len(self.__string):
            if self.__string[i] == "[" or self.__string[i] == "]": self.__snailfish_array.append(self.__string[i])
            if self.__string[i] in integers:
                if self.__string[i+1] in integers:
                    self.__snailfish_array.append(10*int(self.__string[i])+int(self.__string[i+1]))
                    i+=1
                else:
                    self.__snailfish_array.append(int(self.__string[i]))
            i+=1
        if self.__verbose: print(self.__snailfish_array)
        
    def add_new_snailfish_string(self):
        pass
        
    def convert_array_to_snailfish(self):
        converted = str(self.__snailfish_array[0])
        for i in range(1, len(self.__snailfish_array)):
            if self.__snailfish_array[i] != self.__snailfish_array[i-1]:
                converted += ","+str(self.__snailfish_array[i])
            elif type(self.__snailfish_array[i]) == type(1) and \
                 type(self.__snailfish_array[i-1]) == type(1):
                converted += ","+str(self.__snailfish_array[i])
            else:
                converted += str(self.__snailfish_array[i])
        return converted
        
    def __search_next_integer(self, cur_pos, direction):
        '''
        Sucht ab der angegebenen Position nach dem nächstgelegenen Integer
        '''
        if cur_pos >= len(self.__snailfish_array): return None, None
        offset =  1#Gehe eine Position von der aktuellen weg um die nächste Zahl zu finden
        if direction == 1:#Vorwärts
            for i in range(cur_pos+offset, len(self.__snailfish_array)):
                if type(self.__snailfish_array[i]) == type(1):
                    return self.__snailfish_array[i], i

        else:#Rückwärts
            for i in range(cur_pos-offset, 0, -1):
                if type(self.__snailfish_array[i]) == type(1): 
                    return self.__snailfish_array[i], i
        return None, None
    
    def __replace(self, replace_interval, to_insert):
        '''
        Ersetzt die Listenelemente im gegebenen Intervall durch andere Elemente
        '''
        if len(replace_interval) == 1:
            del self.__snailfish_array[replace_interval[0]]
        else:
            del self.__snailfish_array[replace_interval[0]:replace_interval[-1]+1]
        for element in to_insert[::-1]:
            self.__snailfish_array.insert(replace_interval[0], element)
        return self.__snailfish_array
    
    def __explode(self, cur_pos):
        if self.__verbose: print("Explode", cur_pos)
        number_prev, index_prev = self.__search_next_integer(cur_pos, BWD)
        number_1, index_1 = self.__search_next_integer(cur_pos, FWD)
        if self.__verbose: print("prev", number_prev, index_prev)
        if self.__verbose: print("num_1", number_1, index_1)
        #Aktualisiere number_prev
        if index_prev != None:
            number_prev += number_1
            self.__replace([index_prev], [number_prev])
    
        number_2, index_2 = self.__search_next_integer(index_1, FWD)
        number_next, index_next = self.__search_next_integer(index_2, FWD)
        if self.__verbose: print("num_2", number_2, index_2)
        if self.__verbose: print("next", index_next, index_next)        
        #Aktualisiere number_next
        if index_next != None:
            number_next += number_2
            self.__replace([index_next], [number_next])
        if self.__verbose: print("after explode", self.__snailfish_array)   
        #Setze die Klammer auf 0
        if self.__verbose: print(index_1-1, index_2+1)
        if self.__verbose: print(self.__snailfish_array[index_2+1])
        #index_1-1 weil index_1 die erste Zahl ist und -1 die Klammer, welche beide ersetzt werden sollen
        #index_2+2 weil index_2 die zweite Zahl ist und +1 die Klammer
        self.__replace([index_1-1, index_2+1], [0])
        
    def __split(self, cur_pos):
        if self.__verbose: print("Split", cur_pos)
        number_1 = self.__snailfish_array[cur_pos]//2
        number_2 = self.__snailfish_array[cur_pos]//2 + self.__snailfish_array[cur_pos]%2
        if self.__verbose: print(number_1, number_2)
        self.__replace([cur_pos], ["[", number_1, number_2, "]"])
    
    def reduce(self):
        i = 0
        brackets = 0
        while i < len(self.__snailfish_array):
            #Zähle die geöffneten Klammern
            if self.__snailfish_array[i] == PAR_OPEN: 
                brackets += 1
            elif self.__snailfish_array[i] == PAR_CLOSE:
                brackets -= 1
                
            if brackets == 5:
                print("Vor explode:", self.convert_array_to_snailfish())#self.__snailfish_array)
                self.__explode(i)
                print("Nach explode:" , self.convert_array_to_snailfish())#self.__snailfish_array)
                print("\n")
                i = 0
                brackets = 0
                #break
                continue
            if type(self.__snailfish_array[i]) == type(1) and self.__snailfish_array[i] > 9:
                print("Vor split: ", self.convert_array_to_snailfish())
                self.__split(i)
                print("Nach split: ", self.convert_array_to_snailfish())
                print("\n")
                i = 0
                brackets = 0
                #return
                continue
            i += 1
            
    def get_snailfish_array(self):
        return self.__snailfish_array
    
    def test(self):
        print(self.__search_next_integer(6, FWD))
        print(self.__search_next_integer(20, BWD))
        print(self.__replace([4,8], [0]))
        print(self.__replace([16, 17], ["[",4,5,"]"]))
        pass
    
    
#Note: Das Reducen läuft noch nicht korrekt. Mal per Hand machen und nachvollziehen. Die erste Hälft des Testoutputs ist korrekt
str_1 = "[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]"
str_2 = "[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]"
x1 = "["+str_1+","+str_2+"]"
print(x1)
#Erwartes Ergebnis: [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]
x2 = "[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"
day18_test = DAY_18(x1, False)
day18_test.convert_snailfish_to_array()
day18_test.reduce()
print(day18_test.get_snailfish_array())
#day18_test.test()

[[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
Vor explode: [[[[,0,[,4,5,]],[,0,0,]],[[[,4,5,],[,2,6,]],[,9,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]
Nach explode: [[[[,4,0,],[,5,0,]],[[[,4,5,],[,2,6,]],[,9,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]


Vor explode: [[[[,4,0,],[,5,0,]],[[[,4,5,],[,2,6,]],[,9,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]
Nach explode: [[[[,4,0,],[,5,4,]],[[,0,[,7,6,]],[,9,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]


Vor explode: [[[[,4,0,],[,5,4,]],[[,0,[,7,6,]],[,9,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]
Nach explode: [[[[,4,0,],[,5,4,]],[[,7,0,],[,15,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]


Vor split:  [[[[,4,0,],[,5,4,]],[[,7,0,],[,15,5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]
Nach split:  [[[[,4,0,],[,5,4,]],[[,7,0,],[[,7,8,],5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8,8,]]]]]


Vor explode: [[[[,4,0,],[,5,4,]],[[,7,0,],[[,7,8,],5,]]],[,7,[[[,3,7,],[,4,3,]],[[,6,3,],[,8